In [5]:
import numpy as np
import os
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import pickle
import re

# Map for folder names to class labels (16 total classes)
label_mapping = {
    '25_Rotor/25_StatorFault': 0,
    '25_Rotor/50_StatorFault': 1,
    '25_Rotor/75_StatorFault': 2,
    '25_Rotor/NoStatorFault': 3,
    '50_Rotor/25_StatorFault': 4,
    '50_Rotor/50_StatorFault': 5,
    '50_Rotor/75_StatorFault': 6,
    '50_Rotor/NoStatorFault': 7,
    '75_RotorFault/25_StatorFault': 8,
    '75_RotorFault/50_StatorFault': 9,
    '75_RotorFault/75_StatorFault': 10,
    '75_RotorFault/NoStatorFault': 11,
    'NoRotorFault/25_StatorFault': 12,
    'NoRotorFault/50_StatorFault': 13,
    'NoRotorFault/75_StatorFault': 14,
    'NoRotorFault/NoStatorFault': 15
}

# Natural sorting function to handle files like test_1, test_2, ..., test_10
def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]

def load_lvm_data(data_folder):
    data = []
    labels = []

    for main_folder in os.listdir(data_folder):
        main_folder_path = os.path.join(data_folder, main_folder)
        if os.path.isdir(main_folder_path):
            for sub_folder in os.listdir(main_folder_path):
                sub_folder_path = os.path.join(main_folder_path, sub_folder)
                if os.path.isdir(sub_folder_path):
                    class_label = label_mapping.get(f'{main_folder}/{sub_folder}', -1)
                    if class_label == -1:
                        continue  # Skip if folder not in the mapping

                    # Sort the files in increasing order based on their natural numeric order (test_1, test_2, etc.)
                    files = [f for f in os.listdir(sub_folder_path) if f.endswith('.lvm')]
                    files.sort(key=natural_sort_key)  # Apply natural sorting
                    
                    for file in files:
                        file_path = os.path.join(sub_folder_path, file)
                        lvm_data = load_lvm_file(file_path)  # Use the corrected load_lvm_file function
                        data.append(lvm_data)
                        labels.extend([class_label] * lvm_data.shape[0])  # Assign label to all rows

    data = np.vstack(data)  # Stack all the data into a single numpy array
    labels = np.array(labels)  # Convert labels list to numpy array
    return data, labels

def load_lvm_file(file_path):
    # Initialize a list to store the numeric data
    cleaned_data = []
    inconsistent_rows = 0  # Counter for inconsistent rows
    
    print(f"Processing file: {file_path}")  # Debugging statement to show which file is being processed

    # Open the file and read lines
    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()
    except Exception as e:
        print(f"Error opening file {file_path}: {e}")
        return np.array([])  # Return empty if file can't be read
    
    # Find the line where the actual data starts (after the second '***End_of_Header***')
    start_index = 0
    header_count = 0
    for i, line in enumerate(lines):
        if '***End_of_Header***' in line:
            header_count += 1
            if header_count == 2:  # The second '***End_of_Header***' marks the end of headers
                start_index = i + 1
                break
    
    if start_index == 0:
        print(f"Error: No valid data found in {file_path}. Skipping this file.")  # Debugging if header not found
        return np.array([])

    # Process the file starting from the numeric data
    for line_num, line in enumerate(lines[start_index:], start=start_index):
        # Skip empty lines
        line = line.strip()
        if not line:
            continue
        
        # Try to convert each value to a float and then select only columns I1, I2, I3 (after conversion)
        try:
            values = [float(val) for val in line.split('\t') if val.strip()]
            
            # Select only columns 1, 2, and 3 (I1, I2, I3) - ignoring X values and PSD columns
            selected_values = values[1:4]  # Extract I1, I2, I3 columns

            # Only append if the selected values are consistent (i.e., 3 columns)
            if len(selected_values) == 3:
                cleaned_data.append(selected_values)
            else:
                print(f"Inconsistent row (missing I1, I2, I3 columns) at line {line_num + 1} in file {file_path}")
                inconsistent_rows += 1  # Increment counter for inconsistent rows
                
        except ValueError as e:
            print(f"ValueError at line {line_num + 1} in file {file_path}: {e}")
            inconsistent_rows += 1
    
    # Print the number of inconsistent rows at the end
    if inconsistent_rows > 0:
        print(f"Total inconsistent rows skipped in {file_path}: {inconsistent_rows}")
    
    # Convert the cleaned data to a NumPy array
    data = np.array(cleaned_data)
    
    if data.size == 0:
        print(f"No valid data extracted from {file_path}")
    
    return data

# Segment data remains unchanged
def segment_data(data, labels, window_size=1024, step_size=1024):
    num_timesteps, num_features = data.shape
    segmented_data = []
    segmented_labels = []
    
    # Segment data with the sliding window
    for start in range(0, num_timesteps - window_size + 1, step_size):
        end = start + window_size
        segmented_data.append(data[start:end, :])
        
        # Assign the label based on the majority label in the window
        window_label = np.bincount(labels[start:end]).argmax()  # Majority label in the window
        segmented_labels.append(window_label)
    
    segmented_data = np.array(segmented_data)
    segmented_labels = np.array(segmented_labels)
    
    return segmented_data, segmented_labels

def preprocess_data(data, labels, test_size=0.1, val_size=0.2):
    
    # Segment the data into windows of 1024 timesteps
    segmented_data, segmented_labels = segment_data(data, labels)
    
    # Shuffle the segmented samples (important to mix different fault classes)
    indices = np.arange(segmented_data.shape[0])
    np.random.shuffle(indices)
    segmented_data = segmented_data[indices]
    segmented_labels = segmented_labels[indices]
    
    # Split the dataset into training, test, and validation sets
    X_train, X_test, y_train, y_test = train_test_split(segmented_data, segmented_labels, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    
    # Standardize the data (zero mean and unit variance)
    scaler = StandardScaler()
    num_samples, window_size, num_features = X_train.shape
    X_train_reshaped = X_train.reshape(-1, num_features)
    X_val_reshaped = X_val.reshape(-1, num_features)
    X_test_reshaped = X_test.reshape(-1, num_features)
    
    # Fit scaler on training data and transform
    X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(num_samples, window_size, num_features)
    X_val_scaled = scaler.transform(X_val_reshaped).reshape(X_val.shape[0], window_size, num_features)
    X_test_scaled = scaler.transform(X_test_reshaped).reshape(X_test.shape[0], window_size, num_features)
    
    # One-hot encode the labels
    encoder = OneHotEncoder(sparse=False)  # Changed 'sparse_output' to 'sparse'
    y_train_encoded = encoder.fit_transform(y_train.reshape(-1, 1))
    y_val_encoded = encoder.transform(y_val.reshape(-1, 1))
    y_test_encoded = encoder.transform(y_test.reshape(-1, 1))
    
    return X_train_scaled, X_val_scaled, X_test_scaled, y_train_encoded, y_val_encoded, y_test_encoded, scaler


# Save the processed data
def save_data(X_train, X_val, X_test, y_train, y_val, y_test, scaler, filename='preprocessed_data.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump({
            'X_train': X_train,
            'X_val': X_val,
            'X_test': X_test,
            'y_train': y_train,
            'y_val': y_val,
            'y_test': y_test,
            'scaler': scaler
        }, f)

if __name__ == '__main__':
    # Path to the folder containing rotor and stator fault data
    data_folder = r'C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM'
    data, labels = load_lvm_data(data_folder)
    X_train, X_val, X_test, y_train, y_val, y_test, scaler = preprocess_data(data, labels)
    save_data(X_train, X_val, X_test, y_train, y_val, y_test, scaler)


Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_1.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_1.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_1.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_2.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_2.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_2.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_3.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_3.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipp

Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_25.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_26.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_26.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_26.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_27.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_27.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_27.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_28.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Down

Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_52.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_53.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_53.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_53.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_54.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_54.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_54.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_55.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Down

ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_75.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_75.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_76.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_76.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_76.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_77.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_77.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_77.lvm: 1


ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_101.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_101.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_102.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_102.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_102.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_103.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_103.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\25_Rotor\25_StatorFault\test_103

...

Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_399.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_400.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_400.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_400.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_401.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_401.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_401.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_402.lvm
ValueError at line 24 in fi

Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_424.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_425.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_425.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_425.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_426.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_426.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_426.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_427.lvm
ValueError at line 24 in fi

Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_446.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_447.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_447.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_447.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_448.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_448.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_448.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_449.lvm
ValueError at line 24 in fi

Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_471.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_472.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_472.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_472.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_473.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_473.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_473.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_474.lvm
ValueError at line 24 in fi

Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_494.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_495.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_495.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_495.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_496.lvm
ValueError at line 24 in file C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_496.lvm: could not convert string to float: 'X_Value'
Total inconsistent rows skipped in C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_496.lvm: 1
Processing file: C:\Users\SIDDHARTH SINGH\Downloads\RUL_IM\NoRotorFault\NoStatorFault\test_497.lvm
ValueError at line 24 in fi